In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
from math import *
import time

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
# defination of activation function
def activation(x):
    return (torch.sin(x))**3 

In [5]:
# Build ResNet with three blocks
class net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        self.layer1 = torch.nn.Linear(layer_width, layer_width)
        self.layer2 = torch.nn.Linear(layer_width, layer_width)
        self.layer3 = torch.nn.Linear(layer_width, layer_width)
        self.layer4 = torch.nn.Linear(layer_width, layer_width)
        self.layer5 = torch.nn.Linear(layer_width, layer_width)
        self.layer6 = torch.nn.Linear(layer_width, layer_width)
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        y = self.layer_in(x)
        y = y + activation(self.layer2(activation(self.layer1(y)))) # residual block 1
        y = y + activation(self.layer4(activation(self.layer3(y)))) # residual block 2
        y = y + activation(self.layer6(activation(self.layer5(y)))) # residual block 3
        output = self.layer_out(y)
        return output

In [6]:
dimension = 4

In [7]:
# Defination of exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1) # compute norm of x; dim = 1 - > sum by row
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [8]:
# Defination of f(x)
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 1/4*pi**2*torch.sin(pi/2*(1-x_norm)) + 1/2*pi*(dimension-1)/x_norm*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])+ u_temp**3

In [9]:
# Generate points by random
data_size = 1000
def generate_data():
    i = 0
    x = torch.tensor([])
    while x.size()[0] < data_size:
        temp_x = 2*torch.rand(data_size, dimension) - 1
        index = torch.where(torch.norm(temp_x, dim = 1) < 1)
        temp_x = temp_x[index]
        x = torch.cat((x,temp_x),0)
    x = x[1:data_size, :]
    return x.cuda()

In [10]:
# Defination of loss function for DGM
def loss_function(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2    
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1])
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + u_hat**3 - f_temp)**2)/x.size()[0]
    
    Nb = 200
    y = (2*torch.rand(Nb, dimension)-1).cuda()
    y_norm = torch.norm(y, dim = 1).cuda()
    
    for i in range(dimension):
        y[:, i] = y[:, i] / y_norm
    part_2 = (torch.sum((model(y)-u_ex(y))**2)/y.size()[0])
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 

In [11]:
# Parameters initialization by normal distribution for neural networks
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m,nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = net(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

net(
  (layer_in): Linear(in_features=4, out_features=8, bias=True)
  (layer1): Linear(in_features=8, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (layer3): Linear(in_features=8, out_features=8, bias=True)
  (layer4): Linear(in_features=8, out_features=8, bias=True)
  (layer5): Linear(in_features=8, out_features=8, bias=True)
  (layer6): Linear(in_features=8, out_features=8, bias=True)
  (layer_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
# Defination of relative error
def relative_error():
    x = generate_data()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [15]:
traintime = 10000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [16]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = generate_data()
    x.requires_grad = True
    loss = loss_function(x)
    loss.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Dirichlet_nonlinear_sin3.npy", error_save)
np.save("DGM_relative_error_4D_Dirichlet_nonlinear_sin3.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(2219.6931, device='cuda:1')
current relative error is:  tensor(4.5820, device='cuda:1')
current epoch is:  50
current loss is:  tensor(339.4650, device='cuda:1')
current relative error is:  tensor(2.9157, device='cuda:1')
current epoch is:  100
current loss is:  tensor(173.9734, device='cuda:1')
current relative error is:  tensor(2.2563, device='cuda:1')
current epoch is:  150
current loss is:  tensor(122.0598, device='cuda:1')
current relative error is:  tensor(1.9903, device='cuda:1')
current epoch is:  200
current loss is:  tensor(96.6013, device='cuda:1')
current relative error is:  tensor(1.7148, device='cuda:1')
current epoch is:  250
current loss is:  tensor(87.4052, device='cuda:1')
current relative error is:  tensor(1.5649, device='cuda:1')
current epoch is:  300
current loss is:  tensor(63.1700, device='cuda:1')
current relative error is:  tensor(1.4714, device='cuda:1')
current epoch is:  350
current loss is:  tensor(58.4622, dev

current epoch is:  3050
current loss is:  tensor(3.1991, device='cuda:1')
current relative error is:  tensor(0.2281, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(2.7117, device='cuda:1')
current relative error is:  tensor(0.2018, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(2.9457, device='cuda:1')
current relative error is:  tensor(0.1988, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(2.5079, device='cuda:1')
current relative error is:  tensor(0.1775, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(2.3134, device='cuda:1')
current relative error is:  tensor(0.1766, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(2.1692, device='cuda:1')
current relative error is:  tensor(0.1589, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(2.2609, device='cuda:1')
current relative error is:  tensor(0.1555, device='cuda:1')
current epoch is:  3400
current loss is:  tensor(1.8601, devic

current epoch is:  6150
current loss is:  tensor(0.1894, device='cuda:1')
current relative error is:  tensor(0.0379, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(0.1591, device='cuda:1')
current relative error is:  tensor(0.0381, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(0.1684, device='cuda:1')
current relative error is:  tensor(0.0379, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(0.1580, device='cuda:1')
current relative error is:  tensor(0.0389, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(0.1502, device='cuda:1')
current relative error is:  tensor(0.0352, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(0.1403, device='cuda:1')
current relative error is:  tensor(0.0393, device='cuda:1')
current epoch is:  6450
current loss is:  tensor(0.1362, device='cuda:1')
current relative error is:  tensor(0.0382, device='cuda:1')
current epoch is:  6500
current loss is:  tensor(0.1323, devic

current epoch is:  9250
current loss is:  tensor(0.0303, device='cuda:1')
current relative error is:  tensor(0.0215, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(0.0259, device='cuda:1')
current relative error is:  tensor(0.0163, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(0.0245, device='cuda:1')
current relative error is:  tensor(0.0218, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(0.0261, device='cuda:1')
current relative error is:  tensor(0.0144, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(0.0257, device='cuda:1')
current relative error is:  tensor(0.0159, device='cuda:1')
current epoch is:  9500
current loss is:  tensor(0.0234, device='cuda:1')
current relative error is:  tensor(0.0166, device='cuda:1')
current epoch is:  9550
current loss is:  tensor(0.0260, device='cuda:1')
current relative error is:  tensor(0.0183, device='cuda:1')
current epoch is:  9600
current loss is:  tensor(0.0250, devic